In [1]:
# tensorflow와 numpy를 불러옵니다.
import tensorflow as tf
import numpy as np

# 레이블의 데이터는 원-핫 인코딩형태로 구성됩니다.
# 원-핫 인코딩 : 표현하려는 값을 뜻하는 인덱스 원소만 1로 표기하고 나머지 원소는 모두 0으로 채우는 표기법입니다.
# [털, 날개]
x_data = np.array([[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]])
print("x_data : ",np.shape(x_data))
# x_data 는 1X6의 행렬
# 기타 = [1, 0, 0] 포유류 = [0, 1, 0] 조류 = [0, 0, 1]
y_data = np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[1,0,0],[0,0,1]])
print("y_data : ",np.shape(y_data))

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정합니다.
# 편향은 아웃풋의 갯수, 즉 최종 결과값의 분류 갯수인 3으로 설정합니다.
W = tf.Variable(tf.random_uniform([2,3],-1.,1.))
# 가중치 변수 W는 [입력층(특징 수), 출력층(레이블 수)], 편향변수 b는 레이블 수인 3개의 요소를 가진 변수로 설정
b = tf.Variable(tf.zeros([3]))

# 신경망에 가중치 W과 편향 b을 적용합니다
# L = tf.add(tf.matmul(X, W), b) 는 X*W+b를 구현한 것입니다.
L = tf.add(tf.matmul(X, W), b)
# 가중치와 편향을 이용해 계산한 결과 값에
# 텐서플로우에서 기본적으로 제공하는 활성화 함수인 ReLU 함수를 적용합니다
L = tf.nn.relu(L)

# 마지막으로 softmax 함수를 이용하여 출력값을 사용하기 쉽게 만듭니다
# softmax 함수는 다음처럼 결과값을 전체합이 1인 확률로 만들어주는 함수입니다.
# 예) [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]
model = tf.nn.softmax(L)

# 신경망을 최적화하기 위한 비용 함수를 작성합니다.
# 각 개별 결과에 대한 합을 구한 뒤 평균을 내는 방식을 사용합니다.
# 전체 합이 아닌, 개별 결과를 구한 뒤 평균을 내는 방식을 사용하기 위해 axis 옵션을 사용합니다.
# axis 옵션이 없으면 -1.09 처럼 총합인 스칼라값으로 출력됩니다.
#        Y         model         Y * tf.log(model)   reduce_sum(axis=1)
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]
# 즉, 이것은 예측값과 실제값 사이의 확률 분포의 차이를 비용으로 계산한 것이며,
# 이것을 Cross-Entropy 라고 합니다
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model),axis=1))

# 기본적인 경사하강법으로 최적화합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화합니다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
# 신경망 모델 학습
for step in range(100):
  sess.run(train_op, feed_dict={X:x_data, Y:y_data})
  
  # 학습 도중 20번에 한 번씩 손실값을 출력해봅니다.
  if (step + 1) % 50 == 0:
    print(step + 1,sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    print("X : ",sess.run(X, feed_dict={X:x_data}))
    print("W : ",sess.run(W))
    print("b : ",sess.run(b))
    print("L : ",sess.run(L, feed_dict={X: x_data, Y: y_data}))
    print("model : ",sess.run(model, feed_dict={X: x_data, Y: y_data}))
    print("Y : ",sess.run(Y, feed_dict={Y:y_data}))
    
    
#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax 를 이용해 가장 큰 값을 가져옵니다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:',sess.run(prediction, feed_dict={X: x_data}))
print('실제값:',sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction,target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:%.2f' % sess.run(accuracy*100,feed_dict={X:x_data,Y:y_data}))

x_data :  (6, 2)
y_data :  (6, 3)
50 1.1165389
X :  [[0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]]
W :  [[-0.4484563   0.12675464  0.5418512 ]
 [-0.58200765 -0.22083497 -0.952101  ]]
b :  [ 0.          0.00094867 -0.03750202]
L :  [[0.         0.00094867 0.        ]
 [0.         0.12770331 0.5043492 ]
 [0.         0.         0.        ]
 [0.         0.00094867 0.        ]
 [0.         0.00094867 0.        ]
 [0.         0.         0.        ]]
model :  [[0.33322787 0.33354416 0.33322787]
 [0.2637045  0.29962525 0.43667027]
 [0.33333334 0.33333334 0.33333334]
 [0.33322787 0.33354416 0.33322787]
 [0.33322787 0.33354416 0.33322787]
 [0.33333334 0.33333334 0.33333334]]
Y :  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
100 1.1048828
X :  [[0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]]
W :  [[-0.4484563   0.18424524  0.50647944]
 [-0.58200765 -0.22083497 -0.952101  ]]
b :  [ 0.00000e+00  8.34793e-05 -7.28737e-02]
L :  [[0.0000000e+00 8.3479303e-05 0